<a href="https://colab.research.google.com/github/alexcpn/tranformer_learn/blob/main/llama2_overfitting_train2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train a model with a very small data set

For testing this model see https://colab.research.google.com/drive/1IEQz3xRydEbL6b1gbKzGiGoAioa0Tu_w?usp=sharing


In [7]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 huggingface_hub

In [51]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
# lets use the best model in town to check if we get any better
# for NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install -q huggingface_hub
import pprint

In [47]:
# Fine-tuned model name
new_model = "llama-2-7b-miniguanaco"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 50

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 100

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
!huggingface-cli login

In [2]:
#upload files to your colab environment
!wget https://raw.githubusercontent.com/alexcpn/tranformer_learn/main/data/small_3.txt
#!wget https://gist.githubusercontent.com/alexcpn/54e88130f9d186494f1c3ce5e83263b4/raw/7cdf5f93b819024c58a891fc808fbdbe052d0eb1/small_3_mixed.txt

--2023-07-31 03:54:56--  https://raw.githubusercontent.com/alexcpn/tranformer_learn/main/data/small_3.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56513 (55K) [text/plain]
Saving to: ‘small_3.txt’

small_3.txt         100%[===================>]  55.19K  --.-KB/s    in 0.01s   

2023-07-31 03:54:56 (4.22 MB/s) - ‘small_3.txt’ saved [56513/56513]



In [ ]:
# test if drive works
!cp small_3.txt ./drive/MyDrive/models

In [5]:
train_path = 'small_3.txt'


In [6]:
model_name = "meta-llama/Llama-2-7b-chat-hf"

In [9]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling
from transformers import AutoTokenizer

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)


In [18]:
def load_dataset(path,tokenizer):
    dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=path,
          block_size=125)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return dataset,data_collator


In [19]:
train_dataset,data_collator = load_dataset(train_path,tokenizer)

In [48]:
model.train()
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    #group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    data_collator=data_collator,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)


# Train model
trainer.train()

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Step,Training Loss
25,2.479300
50,2.104600
75,1.887600
100,1.701300
125,1.484500
150,1.284000
175,1.043800
200,0.863100
225,0.653900
250,0.496200


TrainOutput(global_step=1400, training_loss=0.31149165187563216, metrics={'train_runtime': 841.2234, 'train_samples_per_second': 6.598, 'train_steps_per_second': 1.664, 'total_flos': 1.4051235520512e+16, 'train_loss': 0.31149165187563216, 'epoch': 50.0})

In [49]:
trainer.save_model() #v100 8 GB GPU RAM

In [52]:
!zip -r llam2-7b-4bit-small3-1400.zip results/checkpoint-1400/


  adding: results/checkpoint-1400/ (stored 0%)
  adding: results/checkpoint-1400/special_tokens_map.json (deflated 73%)
  adding: results/checkpoint-1400/adapter_model.bin (deflated 7%)
  adding: results/checkpoint-1400/README.md (deflated 66%)
  adding: results/checkpoint-1400/adapter_config.json (deflated 43%)
  adding: results/checkpoint-1400/rng_state.pth (deflated 28%)
  adding: results/checkpoint-1400/tokenizer_config.json (deflated 68%)
  adding: results/checkpoint-1400/scheduler.pt (deflated 51%)
  adding: results/checkpoint-1400/trainer_state.json (deflated 86%)
  adding: results/checkpoint-1400/optimizer.pt (deflated 9%)
  adding: results/checkpoint-1400/tokenizer.json (deflated 74%)
  adding: results/checkpoint-1400/training_args.bin (deflated 48%)


In [53]:
!cp llam2-7b-4bit-small3-1400.zip ./drive/MyDrive/models

In [ ]:
from google.colab import runtime
runtime.unassign()

# Test Model

See https://colab.research.google.com/drive/1IEQz3xRydEbL6b1gbKzGiGoAioa0Tu_w?usp=sharing

In [41]:
import pprint
pp = pprint.PrettyPrinter(width=80)

In [ ]:
model.eval()

In [55]:
# First lets check without context what the model gives

prompt = "What is granulation tissue?"
system_message = "You are a helpful Question Answering Assistant.From your training answer the qestion if possible "
prompt_template=f'''[INST] <<SYS>>
{system_message}
<</SYS>>

{prompt} [/INST]'''

inputs = tokenizer(prompt_template, return_tensors="pt")
outputs = model.generate(**inputs.to('cuda') ,max_new_tokens=500)
output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(f"Infer output {output}")
#Output of model trained for 100
outputcont = "".join(output)
parts = outputcont.split("[/INST]", 1)
print(prompt)
pp.pprint(parts[1])

Infer output ['[INST] <<SYS>>\nYou are a helpful Question Answering Assistant.From your training answer the qestion if possible \n<</SYS>>\n\nWhat is granulation tissue? [/INST]  Of course! Granulation tissue is a type of tissue that forms in a wound or surgical bed after the more important structures have been repaired or removed. It is characterized by a mix of immature cells, including fibroblasts, endothelial cells, and leukocytes, and plays a critical role in the healing process by promoting clot stability, filling in the gaps in the broken tissue, and eventually replacing the lost tissue.\nIn more detail, granulation tissue is formed from the following sources:\n\n1. Unstable cells: After an incision or traumatic injury, the exposed tissue is bathed in blood and leukocytes, which help to prevent infection and promote clot formation. As the clot matures, the cells of the clot, particularly the fibroblasts, begin to produce a granulation tissue by converting the fixed connective ti

In [44]:
#Output of model trained for 100
outputcont = "".join(output)
parts = outputcont.split("[/INST]", 1)
print(prompt)
pp.pprint(parts[1])

What is granulation tissue?
("  Of course! I'm glad you think of me as a helpful Question Answering "
 'Assistant. Now, let me answer your question:\n'
 '\n'
 'Granulation tissue, also known as "granulation tissue formation," is a type '
 'of tissue that replaces damaged or lost tissue in the body. It is '
 'characterized by the appearance of small, round, granular masses of new '
 'tissue cells, which gradually consolidate and replace the damaged tissue. '
 'This process is a vital part of the healing process in wounds, and is also '
 'present in various other conditions such as burns, syphilitic and chancroid '
 'tissue damage, and in the aftermath of certain operations.\n'
 '\n'
 'In more detail, the process of granulation tissue formation may be described '
 'as follows: Immediately after an injury, the damaged tissue is invaded by a '
 'group of leucocytes, which include not only the polymorphs, but a large '
 'number of lymphocytes. These leucocytes release a certain amount of '


Granulation tissue is a type of tissue that forms in response to injury or disease. It is characterized by the presence of young capillary loops, supporting cells, and fluids. The tissue is formed in response to the irritation caused by injury or disease, and it plays a vital role in the reparative process. The formation of granulation tissue is a complex process that involves the proliferation and differentiation of various cell types, including fibroblasts, endothelial cells, and immune cells. The tissue is usually fully formed in three to five days, after which it begins to be replaced by cicatricial or scar tissue. The transformation of granulation tissue into scar tissue is effected by the fibroblasts, which become elongated and spindle-shaped and produce a fine network of collagen fibers. The process of healing by primary union is aseptic

In [56]:
#Output of model trained for 1500
outputcont = "".join(output)
parts = outputcont.split("[/INST]", 1)
print(prompt)
pp.pprint(parts[1])

What is granulation tissue?
('  Of course! Granulation tissue is a type of tissue that forms in a wound or '
 'surgical bed after the more important structures have been repaired or '
 'removed. It is characterized by a mix of immature cells, including '
 'fibroblasts, endothelial cells, and leukocytes, and plays a critical role in '
 'the healing process by promoting clot stability, filling in the gaps in the '
 'broken tissue, and eventually replacing the lost tissue.\n'
 'In more detail, granulation tissue is formed from the following sources:\n'
 '\n'
 '1. Unstable cells: After an incision or traumatic injury, the exposed tissue '
 'is bathed in blood and leukocytes, which help to prevent infection and '
 'promote clot formation. As the clot matures, the cells of the clot, '
 'particularly the fibroblasts, begin to produce a granulation tissue by '
 'converting the fixed connective tissue in the wound into a temporary '
 'granulation tissue.\n'
 '\n'
 '2. Proliferation of fibroblas

In [62]:
# Lets check if this overfitted model can also work with previous generic knowledge
prompt = "write python code to scale the Noise figure of -110 dB to 6025 Mhz"
system_message = "You are a programming assistant"
prompt_template=f'''[INST] <<SYS>>
{system_message}
<</SYS>>

{prompt} [/INST]'''

inputs = tokenizer(prompt_template, return_tensors="pt")
outputs = model.generate(**inputs.to('cuda') ,max_new_tokens=200)
output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
outputcont = "".join(output)
parts = outputcont.split("[/INST]", 1)
print(prompt)
pp.pprint(parts[1])

write python code to scale the Noise figure of -110 dB to 6025 Mhz
('  As a programming assistant, I can help you with that! To scale the noise '
 'figure of -110 dB to 6025 MHz, we can use the following formula:\n'
 '\n'
 'noise_figure_in_dB = 10 * log10(frequency_in_MHz / noise_figure_in_dB)\n'
 '\n'
 'Where frequency_in_MHz is the frequency at which we want to calculate the '
 'noise figure in MHz, and noise_figure_in_dB is the desired noise figure in '
 'dB.\n'
 'So, if we want to scale the noise figure of -110 dB to 6025 MHz, we can use '
 'the following code:\n'
 'noise_figure_in_dB = 10 * log10(6025 / -110) = 60.25 dB\n'
 '\n'
 'Note that this formula assumes')


In [79]:
# Not bad - lets check more

# First lets check without context what the model gives

prompt = "Describe Professor Thiersch of Leipsic method of grafting"
system_message = "You are a helpful assistant. From your training data please answer"
prompt_template=f'''[INST] <<SYS>>
{system_message}
<</SYS>>

{prompt} [/INST]'''

inputs = tokenizer(prompt_template, return_tensors="pt")
outputs = model.generate(**inputs.to('cuda') ,max_new_tokens=500)
output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
#print(f"Infer output {output}")


In [65]:
#Output of model trained for 100
outputcont = "".join(output)
parts = outputcont.split("[/INST]", 1)
print(prompt)
pp.pprint(parts[1])

What is the difference between autoplastic grafts and homoplastic grafts?
("  Thank you for asking! I'm here to help. Autoplastic grafts and homoplastic "
 'grafts are both types of tissue grafts used in surgery, but they differ in '
 'the source of the grafting tissue.\n'
 'Autoplastic grafts are tissues taken from the same patient, usually from a '
 'different part of the body. For example, if a patient needs a graft to '
 'repair a damaged blood vessel in their leg, the graft tissue can be taken '
 "from another part of the patient's body, such as their buttock. The "
 'advantage of autoplastic grafts is that they reduce the risk of rejection by '
 "the recipient's immune system, as the tissue is derived from the same "
 'patient.\n'
 'On the other hand, homoplastic grafts are tissues taken from a different '
 'patient, usually from a donor tissue that is similar in structure and '
 'function to the tissue being repaired. For example, if a patient needs a '
 'graft to repair a damag

In [67]:
#Output of model trained for 4000

# prompt = "what is the method introduced by the late Professor Thiersch of Leipsic?"
# system_message = "You are a helpful Question Answering Assistant.From your training answer the qestion if possible "

outputcont = "".join(output)
parts = outputcont.split("[/INST]", 1)
print(prompt)
print(system_message)
pp.pprint(parts[1])

what is the method introduced by the late Professor Thiersch of Leipsic?
('  Thank you for asking! Professor Thiersch of Leipsic (also spelled Leipzig) '
 'introduced a method of surgical treatment for various diseases, including '
 'cancer, which has come to be known as "Thiersch\'s Method." This method '
 'involves the use of a special type of gauze, known as "Thiersch\'s Blue," '
 'which is applied to the surgical site after the operation to promote healing '
 'and prevent infection. The gauze is impregnated with a solution of acetic '
 'acid and other chemicals, which give it antiseptic and stimulant properties. '
 'Thiersch also introduced the use of a special type of surgical glove, known '
 'as a  "Thiersch\'s Glove," which is worn by the surgeon to prevent the risk '
 'of infection. I hope this information is helpful! If you have any further '
 "questions, please don't hesitate to ask.")


In [71]:
outputcont = "".join(output)
parts = outputcont.split("[/INST]", 1)
print(prompt)
print(system_message)
pp.pprint(parts[1])

what is the method introduced by the Professor Thiersch of Leipsic?
You are a helpful Medical Assistant.Based on your training please answer the question or if there is no answer from your training reply so
('  As a Medical Assistant, I am familiar with the method introduced by '
 "Professor Thiersch of Leipsic, which is known as Thiersch's method of bone "
 'grafting. This method involves taking a piece of healthy bone from the ilium '
 'or other bony prominence of the abdominal cavity and transplanting it to the '
 'defective area. The bone is fixed in place with sutures and allowed to heal, '
 'and it is believed that the new bone formed from the graft helps to '
 'consolidate the repair. This method is most successful when the graft is '
 'taken from the ilium because of the aperture of the obtuse marginal sinus, '
 'which aids in promoting healing.  I am not aware of any further developments '
 'or modifications of this method from my training.')


In [74]:
outputcont = "".join(output)
parts = outputcont.split("[/INST]", 1)
print(prompt)
print(system_message)
pp.pprint(parts[1])

what is the method introduced by the Professor Thiersch of Leipsic?
You are a Closed book Question Answering bot.Based on your training data please answer the question or if there is no answer from your training reply so
('  Based on my training data, I am unable to provide an answer to the '
 'question "what is the method introduced by the Professor Thiersch of '
 'Leipsic?" as there is no mention of such a person or their method in my '
 'training data. Professor Thiersch is not a well-known figure in the field of '
 'medicine or surgery, and I cannot generate an answer based on incomplete or '
 'non-existent information. If you could provide more context or details about '
 'Professor Thiersch and his work, I may be able to help you better.')


In [76]:
outputcont = "".join(output)
parts = outputcont.split("[/INST]", 1)
print(prompt)
print(system_message)
pp.pprint(parts[1])

what is the method introduced by the Professor Thiersch of Leipsic?
You are a Closed book Question Answering bot.Based on your training data find the most suitable reference to a question and answer
('  Based on my training data, the most suitable reference to answer the '
 'question "what is the method introduced by the Professor Thiersch of '
 'Leipsic?" is the concept of "septal grafting" or "intercostal grafting." '
 'Professor Thiersch, a German surgeon from Leipzig, introduced this method of '
 'grafting in 1878, and it involves taking grafts of septic tissue from one '
 'part of the body and introducing them into a cavity in another part of the '
 'body, with the aim of restoring healthy tissue and preventing infection. '
 'This method is commonly used in surgery to graft tissue in wounds, defects, '
 'and deformities, and it is considered one of the most important advances in '
 'surgery in the last century.')


In [80]:
outputcont = "".join(output)
parts = outputcont.split("[/INST]", 1)
print(prompt)
print(system_message)
pp.pprint(parts[1])

Describe Professor Thiersch of Leipsic method of grafting
You are a helpful assistant. From your training data please answer
('  Certainly! Professor Thiersch of Leipsic (Leipzig, Germany) is known for '
 'his pioneering work in grafting techniques, and his method of epidermal '
 'grafting is widely recognized as the standard for skin grafting. Here is how '
 "Professor Thiersch's method is typically described:\n"
 '\n'
 'Epidermal Grafting\n'
 '\n'
 "Professor Thiersch's method of epidermal grafting may be outlined as "
 'follows:\n'
 '\n'
 '1. Preparation of the Graft. A piece of healthy epidermis is taken from the '
 'back of the neck and shaved into a thin sheet of uniform thickness, the '
 'graft remaining in this form for 14 to 21 days, when it is expected that the '
 'patient will have recovered from the chloroform anaesthesia. \n'
 '\n'
 "2. Preparation of the Surface. The patient's skin is thoroughly cleaned and "
 'made dry; a circuit of healthy granulations is selected, the 

In [78]:
outputcont = "".join(output)
parts = outputcont.split("[/INST]", 1)
print(prompt)
print(system_message)
pp.pprint(parts[1])

Professor Thiersch of Leipsic 
You are a generative AI; Please complete the rest of the sentence
('  has shown that  generative  or  creative  activity of a higher kind may be '
 "induced in the patient's own tissues. He has successfully transplanted "
 "patches of fresh, undifferentiated cellular tissue from the patient's own "
 'skin or mucous membrane to areas of destruction in the bone, and these '
 'tissues have not only replaced the lost substance with perfect accuracy, but '
 'they have also given evidence of a further process of generation, by the '
 'appearance of small embryonic cells in the newly formed tissue. The process '
 'is similar to the way in which a young healthy plant lets itself be grown on '
 'an elderly plant, and it is possible that in time the transplanted tissue '
 'may become capable of performing functions similar to those of the parent '
 "tissue. Professor Thiersch's process is likely to open up new and important "
 'lines of surgical treatment. \n'
 '\n

Two methods are employed: one in which the epidermis is mainly or exclusively employed epidermis or epithelial grafting; the other, in which the graft consists of the whole thickness of the true skin cutis-grafting.  Epidermis or Epithelial Grafting.  The method introduced by the late Professor Thiersch of Leipsic is that almost universally practised. It consists in transplanting strips of epidermis shaved from the surface of the skin, the razor passing through the tips of the papilla, which appear as tiny red points yielding a moderate ooze of blood. The strips are obtained from the front and lateral aspects of the thigh or upper arm, the skin in those regions being pliable and comparatively free from hairs. They are cut with a sharp hollow-ground razor or with Thiersch's grafting knife, the blade of which is rinsed in alcohol and kept moistened with warm saline solution. The cutting is made easier if the skin is well stretched and kept flat and perfectly steady, the operator's left hand exerting traction on the skin behind, the hands of the assistant on the skin in front, one above and the other below the seat of operation. To ensure uniform strips being cut, the razor is kept parallel with the surface and used with a short, rapid, sawing movement, so that, with a little practice, grafts six or eight inches long by one or two inches broad can readily be cut. The patient is given a general anasthetic, or regional anasthesia is obtained by injections of a solution of one per cent. novocain into the line of the lateral and middle cutaneous nerves; the disinfection of the skin is carried out on the usual lines, any chemical agent being finally got rid of, however, by means of alcohol followed by saline solution. The strips of epidermis wrinkle up on the knife and are directly transferred to the surface, for which they should be made to form a complete carpet, slightly overlapping the edges of the area and of one another; some blunt instrument is used to straighten out the strips, which are then subjected to firm pressure with a pad of gauze to express blood and air-bells and to ensure accurate contact, for this must be as close as that between a postage stamp and the paper to which it is affixed. As a dressing for the grafted area and of that also from which the grafts have been taken, gauze soaked in  liquid paraffin  the patent variety known as  ambrine  is excellent appears to be the best; the gauze should be moistened every other day or so with fresh paraffin, so that, at the end of a week, when the grafts should have united, the gauze can be removed without risk of detaching them.  Dental wax  is another useful type of dressing; as is also  picric acid  solution. Over the gauze, there is applied a thick layer of cotton wool, and the whole dressing is kept in place by a firmly applied bandage, and in the case of the limbs some form of splint should be added to prevent movement. A dressing may be dispensed with altogether, the grafts being protected by a wire cage such as is used after vaccination, but they tend to dry up and come to resemble a scab. When the grafts have healed, it is well to protect them from injury and to prevent them drying up and cracking by the liberal application of lanoline or vaseline. The new skin is at first insensitive and is fixed to the underlying connective tissue or bone, but in course of time (from six weeks onwards) sensation returns and the formation of elastic tissue beneath renders the skin pliant and movable so that it can be pinched up between the finger and thumb.